# Add Transcription Factor Information to Tahoe-100M Genes

This notebook enriches the Tahoe-100M gene list with transcription factor (TF) information:

1. **Is the gene a TF?** - Binary classification
2. **Regulon information** - Target genes regulated by each TF
3. **TF-target relationships** - Mode of regulation (activation/repression)

We'll use:
- **decoupler-py** - Python package for regulatory network analysis
- **CollecTRI** - Literature-curated TF-target database (recommended over DoRothEA)
- **DoRothEA** - Curated TF regulons with confidence levels (A-E)

In [21]:
# Import required libraries
import pandas as pd
import numpy as np
from pathlib import Path
import decoupler as dc

print(f"decoupler version: {dc.__version__}")

decoupler version: 1.9.0


## 1. Load Tahoe-100M Gene List

In [25]:
# Load the gene list we created earlier
genes_df = pd.read_csv('../data/tahoe_100m_genes.csv')

print(f"Total genes: {len(genes_df)}")
print(f"Columns: {genes_df.columns.tolist()}")
genes_df.head()

Total genes: 62710
Columns: ['gene_symbol', 'ensembl_id', 'token_id']


,gene_symbol,ensembl_id,token_id
0,TSPAN6,ENSG00000000003,3
1,TNMD,ENSG00000000005,4
2,DPM1,ENSG00000000419,5
3,SCYL3,ENSG00000000457,6
4,C1orf112,ENSG00000000460,7


## 2. Load TF Databases

### 2.1 CollecTRI (Recommended)
CollecTRI is a comprehensive literature-curated regulatory network with:
- High coverage of TF-target interactions
- Mode of regulation (positive/negative)

In [26]:
# Load CollecTRI database
print("Loading CollecTRI TF-target database...")
collectri = dc.get_collectri(organism='human')

print(f"\nCollecTRI shape: {collectri.shape}")
print(f"Columns: {collectri.columns.tolist()}")
print(f"\nUnique TFs: {collectri['source'].nunique()}")
print(f"Unique targets: {collectri['target'].nunique()}")
print(f"Total interactions: {len(collectri)}")

collectri.head(10)

Loading CollecTRI TF-target database...

CollecTRI shape: (64516, 4)
Columns: ['source', 'target', 'weight', 'pmid']

Unique TFs: 1163
Unique targets: 6628
Total interactions: 64516


,source,target,weight,pmid
0,MYC,TERT,1,10022128;10491298;10606235;10637317;10723141;1...
1,SPI1,BGLAP,1,10022617
2,AP1,JUN,1,10022869;10037172;10208431;10366004;11281649;1...
3,AP1,JUN,1,10022869;10037172;10208431;10366004;11281649;1...
4,AP1,JUN,1,10022869;10037172;10208431;10366004;11281649;1...
5,AP1,JUN,1,10022869;10037172;10208431;10366004;11281649;1...
6,AP1,JUN,1,10022869;10037172;10208431;10366004;11281649;1...
7,AP1,JUN,1,10022869;10037172;10208431;10366004;11281649;1...
8,AP1,JUN,1,10022869;10037172;10208431;10366004;11281649;1...
9,AP1,JUN,1,10022869;10037172;10208431;10366004;11281649;1...


### 2.2 DoRothEA
DoRothEA provides regulons with confidence levels (A = highest, E = lowest)

In [8]:
# Load DoRothEA database (confidence levels A-D, excluding E)
print("Loading DoRothEA TF-target database...")
dorothea = dc.get_dorothea(organism='human', levels=['A', 'B', 'C', 'D'])

print(f"\nDoRothEA shape: {dorothea.shape}")
print(f"Columns: {dorothea.columns.tolist()}")
print(f"\nUnique TFs: {dorothea['source'].nunique()}")
print(f"Unique targets: {dorothea['target'].nunique()}")
print(f"Total interactions: {len(dorothea)}")

# Show confidence level distribution
print(f"\nConfidence level distribution:")
print(dorothea['confidence'].value_counts().sort_index())

dorothea.head(10)

Loading DoRothEA TF-target database...

DoRothEA shape: (276562, 4)
Columns: ['source', 'target', 'weight', 'confidence']

Unique TFs: 643
Unique targets: 18564
Total interactions: 276562

Confidence level distribution:
confidence
A      6080
B      9037
C     17169
D    244276
Name: count, dtype: Int64


,source,target,weight,confidence
0,MYC,TERT,1.0,A
1,TP53,TYMS,-1.0,A
2,HIF1A,TF,1.0,A
3,EGR1,CD19,1.0,A
4,TFAP2A,HSD17B1,-1.0,A
5,THRB,POR,1.0,A
6,JUN,ICAM1,1.0,A
7,STAT2,IRF1,1.0,A
8,PPARA,APOC3,-1.0,A
9,TAL1,DOCK9,1.0,A


## 3. Identify Transcription Factors

A gene is a TF if it appears as a 'source' (regulator) in either database.

In [9]:
# Get unique TFs from both databases
collectri_tfs = set(collectri['source'].unique())
dorothea_tfs = set(dorothea['source'].unique())

# Union of TFs from both databases
all_tfs = collectri_tfs | dorothea_tfs

print(f"CollecTRI TFs: {len(collectri_tfs)}")
print(f"DoRothEA TFs: {len(dorothea_tfs)}")
print(f"Combined unique TFs: {len(all_tfs)}")
print(f"TFs in both databases: {len(collectri_tfs & dorothea_tfs)}")

CollecTRI TFs: 1163
DoRothEA TFs: 643
Combined unique TFs: 1204
TFs in both databases: 602


In [10]:
# Check how many Tahoe genes are TFs
tahoe_genes = set(genes_df['gene_symbol'].unique())

tf_in_tahoe = all_tfs & tahoe_genes
print(f"\nTFs present in Tahoe-100M gene list: {len(tf_in_tahoe)}")
print(f"Percentage of Tahoe genes that are TFs: {len(tf_in_tahoe)/len(tahoe_genes)*100:.2f}%")


TFs present in Tahoe-100M gene list: 1202
Percentage of Tahoe genes that are TFs: 1.92%


## 4. Build Regulon Information

For each TF, we'll collect:
- Number of target genes
- Target gene list
- Number of activated targets
- Number of repressed targets

In [11]:
# Build regulon info from CollecTRI
def build_regulon_info(network_df):
    """
    Build regulon information for each TF.
    
    Returns a DataFrame with:
    - tf: Transcription factor name
    - n_targets: Number of target genes
    - n_activated: Number of positively regulated targets
    - n_repressed: Number of negatively regulated targets
    - targets: Comma-separated list of target genes
    - activated_targets: List of activated targets
    - repressed_targets: List of repressed targets
    """
    regulon_info = []
    
    for tf in network_df['source'].unique():
        tf_network = network_df[network_df['source'] == tf]
        
        # Get targets
        all_targets = tf_network['target'].tolist()
        
        # Get mode of regulation (weight column: positive = activation, negative = repression)
        if 'weight' in tf_network.columns:
            activated = tf_network[tf_network['weight'] > 0]['target'].tolist()
            repressed = tf_network[tf_network['weight'] < 0]['target'].tolist()
        else:
            activated = all_targets
            repressed = []
        
        regulon_info.append({
            'tf': tf,
            'n_targets': len(all_targets),
            'n_activated': len(activated),
            'n_repressed': len(repressed),
            'targets': ','.join(sorted(all_targets)),
            'activated_targets': ','.join(sorted(activated)),
            'repressed_targets': ','.join(sorted(repressed))
        })
    
    return pd.DataFrame(regulon_info)

# Build regulon info from CollecTRI
print("Building regulon information from CollecTRI...")
collectri_regulons = build_regulon_info(collectri)
collectri_regulons = collectri_regulons.add_prefix('collectri_')
collectri_regulons = collectri_regulons.rename(columns={'collectri_tf': 'gene_symbol'})

print(f"CollecTRI regulons: {len(collectri_regulons)}")
collectri_regulons.head()

Building regulon information from CollecTRI...
CollecTRI regulons: 1163


,gene_symbol,collectri_n_targets,collectri_n_activated,collectri_n_repressed,collectri_targets,collectri_activated_targets,collectri_repressed_targets
0,MYC,891,804,87,"ABCA4,ABCB1,ABCB6,ABI2,ABL1,ACADM,ACE,ACHE,ACO...","ABCA4,ABCB1,ABCB6,ABL1,ACADM,ACE,ACHE,ACO2,ACO...","ABI2,ACSL4,ALB,ALDH9A1,BCL2L1,BRD2,BRD7,CASP8,..."
1,SPI1,349,328,21,"A2M,AAR2,ABL1,ACAN,ACP5,ADAR,ADGRE4P,AICDA,AKT...","A2M,AAR2,ABL1,ACAN,ACP5,ADAR,ADGRE4P,AICDA,AKT...","ANXA1,BCL6,CD68,GATA1,GATA2,HBB,HES1,HLA-DQB1,..."
2,AP1,12105,11309,796,"A2M,A2M,AANAT,AANAT,ABCA1,ABCB1,ABCB1,ABCB1,AB...","A2M,A2M,AANAT,AANAT,ABCB1,ABCB11,ABCB11,ABCB11...","ABCA1,ABCB1,ABCB1,ABCB1,ABCB1,ABCB1,ABCB1,ABCB..."
3,SMAD3,191,153,38,"ACHE,ACTA1,ACTA2,ACTG2,AGTR1,AHR,ALPI,ANGPTL2,...","ACHE,ACTA1,ACTA2,ACTG2,AGTR1,AHR,ALPI,ANGPTL2,...","ASIC3,BAX,BCL2,CDC25A,CDH1,CIITA,CLDN3,CSN2,CY..."
4,SMAD4,143,126,17,"AGR2,AGTR1,AHR,ALPI,ANKRD1,APOC3,APP,AR,ATOH8,...","AGR2,AGTR1,AHR,ALPI,ANKRD1,APOC3,APP,ATOH8,BAM...","AR,BMP6,BTRC,CCL15,CDH1,CLDN1,CLDN3,DACH1,DPYS..."


In [12]:
# Build regulon info from DoRothEA
print("Building regulon information from DoRothEA...")
dorothea_regulons = build_regulon_info(dorothea)
dorothea_regulons = dorothea_regulons.add_prefix('dorothea_')
dorothea_regulons = dorothea_regulons.rename(columns={'dorothea_tf': 'gene_symbol'})

print(f"DoRothEA regulons: {len(dorothea_regulons)}")
dorothea_regulons.head()

Building regulon information from DoRothEA...
DoRothEA regulons: 643


,gene_symbol,dorothea_n_targets,dorothea_n_activated,dorothea_n_repressed,dorothea_targets,dorothea_activated_targets,dorothea_repressed_targets
0,MYC,1363,1322,41,"ABCB10,ABCB6,ABCB9,ABCC4,ABR,ACADM,ACAP3,ACE,A...","ABCB10,ABCB6,ABCB9,ABCC4,ABR,ACADM,ACAP3,ACE,A...","ALDH9A1,BRD7,CASP8,CAST,CD33,CDKN1A,CDKN1B,CDK..."
1,TP53,2740,2630,110,"A1CF,AADACL4,AAK1,AARS2,AASDHPPT,ABCA12,ABCA13...","A1CF,AADACL4,AAK1,AARS2,AASDHPPT,ABCA12,ABCA13...","ABCB1,AFP,AKT1,AR,BAG1,BCL2,BCL2L1,BCL3,BIRC5,..."
2,HIF1A,1210,1200,10,"ABCA1,ABCA13,ABCA4,ABCB1,ABCG2,ABHD17C,ABLIM2,...","ABCA1,ABCA13,ABCA4,ABCB1,ABCG2,ABHD17C,ABLIM2,...","ACE2,ANGPT1,APEX1,BID,CDH1,KCNA5,LGALS1,RECK,S..."
3,EGR1,10666,10630,36,"A1BG,A3GALT2,A4GALT,AAAS,AACS,AADACL2,AADACL4,...","A1BG,A3GALT2,A4GALT,AAAS,AACS,AADACL2,AADACL4,...","ADRB1,AR,ATP2A2,BCL2,BNIP3,CASP1,CAVIN1,CDC20,..."
4,TFAP2A,758,748,10,"ABCB1,ABCC3,ABCC5,ABCD1,ABCD2,ABCG1,ACHE,ACOX3...","ABCB1,ABCC3,ABCC5,ABCD1,ABCD2,ABCG1,ACOX3,ACOX...","ACHE,BCL2,F2R,HGF,HOXA4,HSD17B1,KLF12,MCAM,MUC..."


## 5. Add "Is Target" Information

For genes that are NOT TFs, we can still add whether they are targets of TFs.

In [13]:
# Get genes that are targets of TFs
collectri_targets = set(collectri['target'].unique())
dorothea_targets = set(dorothea['target'].unique())
all_targets = collectri_targets | dorothea_targets

print(f"CollecTRI targets: {len(collectri_targets)}")
print(f"DoRothEA targets: {len(dorothea_targets)}")
print(f"Combined unique targets: {len(all_targets)}")

# How many Tahoe genes are targets?
targets_in_tahoe = all_targets & tahoe_genes
print(f"\nTarget genes in Tahoe-100M: {len(targets_in_tahoe)}")
print(f"Percentage: {len(targets_in_tahoe)/len(tahoe_genes)*100:.2f}%")

CollecTRI targets: 6628
DoRothEA targets: 18564
Combined unique targets: 18633

Target genes in Tahoe-100M: 18465
Percentage: 29.45%


In [14]:
# For each gene, count how many TFs regulate it
def count_regulators(gene, network_df):
    """Count how many TFs regulate a given gene."""
    regulators = network_df[network_df['target'] == gene]['source'].unique()
    return len(regulators)

# Create a lookup for number of regulators per gene
print("Counting regulators for each target gene (CollecTRI)...")
collectri_regulator_counts = collectri.groupby('target')['source'].nunique().reset_index()
collectri_regulator_counts.columns = ['gene_symbol', 'collectri_n_regulators']

print("Counting regulators for each target gene (DoRothEA)...")
dorothea_regulator_counts = dorothea.groupby('target')['source'].nunique().reset_index()
dorothea_regulator_counts.columns = ['gene_symbol', 'dorothea_n_regulators']

print(f"\nGenes with CollecTRI regulator info: {len(collectri_regulator_counts)}")
print(f"Genes with DoRothEA regulator info: {len(dorothea_regulator_counts)}")

Counting regulators for each target gene (CollecTRI)...
Counting regulators for each target gene (DoRothEA)...

Genes with CollecTRI regulator info: 6628
Genes with DoRothEA regulator info: 18564


## 6. Merge All Information

In [28]:
# Start with the original gene list
genes_enriched = genes_df.copy()

# Add binary TF flags
genes_enriched['is_tf_collectri'] = genes_enriched['gene_symbol'].isin(collectri_tfs)
genes_enriched['is_tf_dorothea'] = genes_enriched['gene_symbol'].isin(dorothea_tfs)
genes_enriched['is_tf'] = genes_enriched['gene_symbol'].isin(all_tfs)

# Add target flags
genes_enriched['is_target_collectri'] = genes_enriched['gene_symbol'].isin(collectri_targets)
genes_enriched['is_target_dorothea'] = genes_enriched['gene_symbol'].isin(dorothea_targets)
genes_enriched['is_target'] = genes_enriched['gene_symbol'].isin(all_targets)

print(f"Genes enriched with TF flags")
print(f"TFs: {genes_enriched['is_tf'].sum()}")
print(f"Targets: {genes_enriched['is_target'].sum()}")

Genes enriched with TF flags
TFs: 1202
Targets: 18465


In [16]:
# Merge CollecTRI regulon info (for TFs)
genes_enriched = genes_enriched.merge(
    collectri_regulons, 
    on='gene_symbol', 
    how='left'
)

# Merge DoRothEA regulon info (for TFs)
genes_enriched = genes_enriched.merge(
    dorothea_regulons, 
    on='gene_symbol', 
    how='left'
)

# Merge regulator counts (for targets)
genes_enriched = genes_enriched.merge(
    collectri_regulator_counts, 
    on='gene_symbol', 
    how='left'
)

genes_enriched = genes_enriched.merge(
    dorothea_regulator_counts, 
    on='gene_symbol', 
    how='left'
)

# Fill NaN values
for col in ['collectri_n_targets', 'collectri_n_activated', 'collectri_n_repressed',
            'dorothea_n_targets', 'dorothea_n_activated', 'dorothea_n_repressed',
            'collectri_n_regulators', 'dorothea_n_regulators']:
    if col in genes_enriched.columns:
        genes_enriched[col] = genes_enriched[col].fillna(0).astype(int)

print(f"\nEnriched gene dataframe shape: {genes_enriched.shape}")
print(f"Columns: {genes_enriched.columns.tolist()}")


Enriched gene dataframe shape: (62710, 23)
Columns: ['gene_symbol', 'ensembl_id', 'token_id', 'is_tf_collectri', 'is_tf_dorothea', 'is_tf', 'is_target_collectri', 'is_target_dorothea', 'is_target', 'collectri_n_targets', 'collectri_n_activated', 'collectri_n_repressed', 'collectri_targets', 'collectri_activated_targets', 'collectri_repressed_targets', 'dorothea_n_targets', 'dorothea_n_activated', 'dorothea_n_repressed', 'dorothea_targets', 'dorothea_activated_targets', 'dorothea_repressed_targets', 'collectri_n_regulators', 'dorothea_n_regulators']


In [17]:
# View some TFs
print("Sample TFs with regulon info:")
tf_sample = genes_enriched[genes_enriched['is_tf']][[
    'gene_symbol', 'ensembl_id', 'is_tf', 
    'collectri_n_targets', 'collectri_n_activated', 'collectri_n_repressed',
    'dorothea_n_targets'
]].head(20)
tf_sample

Sample TFs with regulon info:


,gene_symbol,ensembl_id,is_tf,collectri_n_targets,collectri_n_activated,collectri_n_repressed,dorothea_n_targets
9,NFYA,ENSG00000001167,True,95,90,5,518
64,ARX,ENSG00000004848,True,27,20,7,4
77,HOXA11,ENSG00000005073,True,8,4,4,2
80,MEOX1,ENSG00000005102,True,9,7,2,0
92,CREBBP,ENSG00000005339,True,36,35,1,0
100,KMT2E,ENSG00000005483,True,4,4,0,0
102,SOX8,ENSG00000005513,True,4,3,1,1
104,ZNF195,ENSG00000005801,True,1,1,0,0
110,ZFX,ENSG00000005889,True,5,4,1,442
119,YBX2,ENSG00000006047,True,1,1,0,0


In [18]:
# View some target genes
print("Sample target genes with regulator info:")
target_sample = genes_enriched[
    (genes_enriched['is_target']) & (~genes_enriched['is_tf'])
][[
    'gene_symbol', 'ensembl_id', 'is_target',
    'collectri_n_regulators', 'dorothea_n_regulators'
]].head(20)
target_sample

Sample target genes with regulator info:


,gene_symbol,ensembl_id,is_target,collectri_n_regulators,dorothea_n_regulators
0,TSPAN6,ENSG00000000003,True,0,2
1,TNMD,ENSG00000000005,True,3,2
2,DPM1,ENSG00000000419,True,0,4
3,SCYL3,ENSG00000000457,True,0,14
5,FGR,ENSG00000000938,True,0,20
6,CFH,ENSG00000000971,True,6,6
7,FUCA2,ENSG00000001036,True,0,8
8,GCLC,ENSG00000001084,True,21,59
10,STPG1,ENSG00000001460,True,0,24
11,NIPAL3,ENSG00000001461,True,0,12


## 7. Summary Statistics

In [19]:
print("="*60)
print("SUMMARY STATISTICS")
print("="*60)

print(f"\nTotal genes in Tahoe-100M: {len(genes_enriched)}")

print(f"\n--- Transcription Factors ---")
print(f"TFs (any database): {genes_enriched['is_tf'].sum()} ({genes_enriched['is_tf'].mean()*100:.2f}%)")
print(f"TFs (CollecTRI): {genes_enriched['is_tf_collectri'].sum()}")
print(f"TFs (DoRothEA): {genes_enriched['is_tf_dorothea'].sum()}")

print(f"\n--- Target Genes ---")
print(f"Targets (any database): {genes_enriched['is_target'].sum()} ({genes_enriched['is_target'].mean()*100:.2f}%)")
print(f"Targets (CollecTRI): {genes_enriched['is_target_collectri'].sum()}")
print(f"Targets (DoRothEA): {genes_enriched['is_target_dorothea'].sum()}")

print(f"\n--- Genes with no TF annotation ---")
no_annotation = (~genes_enriched['is_tf']) & (~genes_enriched['is_target'])
print(f"Genes not TF and not target: {no_annotation.sum()} ({no_annotation.mean()*100:.2f}%)")

SUMMARY STATISTICS

Total genes in Tahoe-100M: 62710

--- Transcription Factors ---
TFs (any database): 1202 (1.92%)
TFs (CollecTRI): 1161
TFs (DoRothEA): 643

--- Target Genes ---
Targets (any database): 18465 (29.45%)
Targets (CollecTRI): 6614
Targets (DoRothEA): 18399

--- Genes with no TF annotation ---
Genes not TF and not target: 44243 (70.55%)


In [34]:
# Top TFs by number of targets
print("\nTop 20 TFs by number of targets (CollecTRI):")
top_tfs = genes_enriched[genes_enriched['is_tf']].nlargest(20, 'collectri_n_targets')[[
    'gene_symbol', 'collectri_n_targets', 'collectri_n_activated', 'collectri_n_repressed'
]]
print(top_tfs.to_string(index=False))


Top 20 TFs by number of targets (CollecTRI):


KeyError: 'collectri_n_targets'

## 8. Save Enriched Gene List

In [ ]:
# Define output paths
output_dir = Path('../data')

# Save full version (with target lists - larger file)
full_output = output_dir / 'tahoe_100m_genes_tf_enriched_full.csv'
genes_enriched.to_csv(full_output, index=False)
print(f"Full enriched gene list saved to: {full_output}")
print(f"File size: {full_output.stat().st_size / (1024*1024):.2f} MB")

# Save compact version (without target lists - smaller file)
compact_cols = [
    'gene_symbol', 'ensembl_id', 'token_id',
    'is_tf', 'is_tf_collectri', 'is_tf_dorothea',
    'is_target', 'is_target_collectri', 'is_target_dorothea',
    'collectri_n_targets', 'collectri_n_activated', 'collectri_n_repressed',
    'dorothea_n_targets', 'dorothea_n_activated', 'dorothea_n_repressed',
    'collectri_n_regulators', 'dorothea_n_regulators'
]
genes_compact = genes_enriched[[c for c in compact_cols if c in genes_enriched.columns]]

compact_output = output_dir / 'tahoe_100m_genes_tf_enriched.csv'
genes_compact.to_csv(compact_output, index=False)
print(f"\nCompact enriched gene list saved to: {compact_output}")
print(f"File size: {compact_output.stat().st_size / 1024:.1f} KB")

In [ ]:
# Verify saved files
print("\nVerification:")
df_check = pd.read_csv(compact_output)
print(f"Rows: {len(df_check)}")
print(f"Columns: {df_check.columns.tolist()}")
df_check.head()

## 9. Export TF-only List

In [ ]:
# Export just the TFs with their regulon info
tf_only = genes_enriched[genes_enriched['is_tf']].copy()

tf_output = output_dir / 'tahoe_100m_transcription_factors.csv'
tf_only.to_csv(tf_output, index=False)

print(f"TF-only list saved to: {tf_output}")
print(f"Total TFs: {len(tf_only)}")
print(f"File size: {tf_output.stat().st_size / (1024*1024):.2f} MB")

In [ ]:
print("\n" + "="*60)
print("COMPLETE!")
print("="*60)
print(f"\nFiles created:")
print(f"1. {compact_output.name} - Compact TF-enriched gene list")
print(f"2. {full_output.name} - Full version with target lists")
print(f"3. {tf_output.name} - TFs only with regulon info")